<a href="https://colab.research.google.com/github/WilliamAshbee/DataminingAssignments/blob/main/DM_explore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn as sk
import pandas as pd


In [2]:
#!wget http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/cleve.mod

In [3]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/cleve.mod'
df1 = pd.read_csv(url,header=None,skiprows=(i for i in range(20)),delim_whitespace=True,error_bad_lines=False)

url2 = 'http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
df_clev = pd.read_csv(url2,header=None,error_bad_lines=False)

In [4]:
print(df1.shape,df_clev.shape)

(303, 15) (303, 14)


In [5]:
pain = {'angina': 1, 'asympt': 4,'notang':2,'abnang':3}
gender = {'male':1,'fem':0}
bsl20 = {'true':1,'fal':0}
restingecg = {'norm':0,'abn':1, 'hyp':2}
exindang = {'true':1,'fal':0}
slope = {'up':1, 'flat':2, 'down':3}
thal = {'norm':1, 'fix':2, 'rev':3}
state= {'buff':1,'sick':-1}
code = {'H':0,'S1':1,'S2':2,'S3':3,'S4':4}
eleventh = {'?':-10000}
twelth = {'?':-10000}
df2 = df1.replace({1: gender,2:pain,5:bsl20,6:restingecg,8:exindang,10:slope,11:eleventh,12:{**thal, **twelth},13:state,14:code})


In [6]:
for col in df2.columns:
    print(type(col),col)#column names i assume

<class 'int'> 0
<class 'int'> 1
<class 'int'> 2
<class 'int'> 3
<class 'int'> 4
<class 'int'> 5
<class 'int'> 6
<class 'int'> 7
<class 'int'> 8
<class 'int'> 9
<class 'int'> 10
<class 'int'> 11
<class 'int'> 12
<class 'int'> 13
<class 'int'> 14


In [7]:
df2.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
298,48.0,1,2,124.0,255.0,1,0,175.0,0,0.0,1,2.0,1,1,0
299,57.0,1,4,132.0,207.0,0,0,168.0,1,0.0,1,0.0,3,1,0
300,49.0,1,2,118.0,149.0,0,2,126.0,0,0.8,1,3.0,1,-1,1
301,74.0,0,3,120.0,269.0,0,2,121.0,1,0.2,1,1.0,1,1,0
302,54.0,0,2,160.0,201.0,0,0,163.0,0,0.0,1,1.0,1,1,0


In [8]:
import numpy as np

df2n = df2.to_numpy().astype(float)

df2n.shape


(303, 15)

In [9]:
questionmark = {'?':'-10000'}

dfcn = df_clev
dfcn = dfcn.replace({11:questionmark,12:questionmark})
dfcn = dfcn.to_numpy().astype(float)

print(np.sum(dfcn=='?'),np.sum(df2n=='?'))

0 0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys


In [10]:
print(dfcn[0,:].astype(float))
print(df2n[0,:])
print(np.sum(dfcn[:,:13].astype(float)-df2n[:,:13].astype(float)))

[ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   6.    0. ]
[ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   2.    1.    0. ]
11071.0


In [11]:
print(np.min(dfcn[:,13]),np.max(dfcn[:,13]))

0.0 4.0


In [12]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression
import os
#change here
y = dfcn[:,13]

X = dfcn[:,:13]

LR = LogisticRegression(max_iter = 100000,random_state=0, solver='lbfgs', multi_class='auto').fit(X, y)
round(LR.score(X,y), 5)


0.62046

In [13]:
#https://stackoverflow.com/questions/17114904/python-pandas-replacing-strings-in-dataframe-with-numbers
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html

In [14]:
out = LR.predict(X)

In [15]:
np.mean(out == y)

0.6204620462046204

In [16]:
from sklearn import svm
clf = svm.SVC(decision_function_shape='ovo')
print(clf.kernel)
clf = clf.fit(X, y)
out = clf.predict(X)
print(np.mean(out==y))

rbf
0.5412541254125413


In [17]:
C_2d_range = [1e-2, 1, 1e2]
gamma_2d_range = [1e-1, 1, 1e1]
classifiers = []
for C in C_2d_range:
    for gamma in gamma_2d_range:
        clf = svm.SVC(C=C, gamma=gamma,decision_function_shape='ovo')
        clf.fit(X, y)
        out = clf.predict(X)
        if np.mean(y==out) > .6:
          print(np.mean(y==out),C,gamma)
        classifiers.append((C, gamma, clf,np.mean(y==out)))

clf = svm.SVC(C=1, gamma=.1,decision_function_shape='ovo')
clf.fit(X, y)
clf.predict(X)

1.0 1 0.1
1.0 1 1
1.0 1 10.0
1.0 100.0 0.1
1.0 100.0 1
1.0 100.0 10.0


array([0., 2., 1., 0., 0., 0., 3., 0., 2., 1., 0., 0., 2., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 1., 3., 4., 0., 0., 0., 0., 3., 0., 2., 1., 0.,
       0., 0., 3., 1., 3., 0., 4., 0., 0., 0., 1., 4., 0., 4., 0., 0., 0.,
       0., 2., 0., 1., 1., 1., 1., 0., 0., 2., 0., 1., 0., 2., 2., 1., 0.,
       2., 1., 0., 3., 1., 1., 1., 0., 1., 0., 0., 3., 0., 0., 0., 3., 0.,
       0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 1., 2., 3., 0., 0., 0., 0.,
       0., 0., 3., 0., 2., 1., 2., 3., 1., 1., 0., 2., 2., 0., 0., 0., 3.,
       2., 3., 4., 0., 3., 1., 0., 3., 3., 0., 0., 0., 0., 0., 0., 0., 0.,
       4., 3., 1., 0., 0., 1., 0., 1., 0., 1., 4., 0., 0., 0., 0., 0., 0.,
       4., 3., 1., 1., 1., 2., 0., 0., 4., 0., 0., 0., 0., 0., 0., 1., 0.,
       3., 0., 1., 0., 4., 1., 0., 1., 0., 0., 3., 2., 0., 0., 1., 0., 0.,
       2., 1., 2., 0., 3., 1., 2., 0., 3., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 3., 3., 3., 0., 1., 0., 4., 0., 3., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 3., 1., 0.

In [18]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=30000,learning_rate_init = .00001).fit(X, y)
clf.predict(X)
clf.score(X, y)


0.594059405940594

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
#X, y = make_classification(n_samples=1000, n_features=4,
#...                            n_informative=2, n_redundant=0,
#...                            random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=7, random_state=0)
clf.fit(X, y)
print(clf.score(X,y))


0.9702970297029703


In [20]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB().fit(X, y)
gnb_score = gnb.score(X,y)
print(gnb_score)

0.18151815181518152


In [21]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(X, y)
print(neigh.score(X,y))

0.6567656765676567


In [22]:
#https://medium.com/@rrfd/boosting-bagging-and-stacking-ensemble-methods-with-sklearn-and-mlens-a455c0c982de

In [23]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score

clf = KNeighborsClassifier()
clf.fit(X,y)
bagging_clf = BaggingClassifier(clf, max_samples=0.4, max_features=4, random_state=0)
bagging_scores = cross_val_score(bagging_clf, X, y, cv=10, n_jobs=-1)
print(bagging_scores)
knnscores = cross_val_score(clf, X, y, cv=10, n_jobs=-1)
print(knnscores)

[0.5483871  0.5483871  0.58064516 0.6        0.56666667 0.66666667
 0.56666667 0.53333333 0.53333333 0.53333333]
[0.5483871  0.48387097 0.51612903 0.56666667 0.53333333 0.4
 0.53333333 0.53333333 0.33333333 0.53333333]


In [24]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(base_estimator=None)
clf.fit(X,y)
clf.score(X,y)
cross_val_score(clf, X, y, cv=10, n_jobs=-1)

array([0.67741935, 0.58064516, 0.5483871 , 0.66666667, 0.53333333,
       0.46666667, 0.5       , 0.43333333, 0.46666667, 0.46666667])

In [25]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
grad_boost = GradientBoostingClassifier()
cross_val_score(grad_boost, X, y, cv=10, n_jobs=-1)

array([0.61290323, 0.51612903, 0.61290323, 0.56666667, 0.63333333,
       0.46666667, 0.46666667, 0.63333333, 0.53333333, 0.53333333])

In [26]:
#
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from sklearn.linear_model import RidgeClassifier
rf = RandomForestClassifier()
et = ExtraTreesClassifier()
knn = KNeighborsClassifier()
svc = svm.SVC()
rg = RidgeClassifier()

clf = [rf, et, knn, svc, rg]
eclf = VotingClassifier(estimators=[('Random Forests', rf), ('Extra Trees', et), ('KNeighbors', knn), ('SVC', svc), ('Ridge Classifier', rg)], voting='hard')
for clf, label in zip([rf, et, knn, svc, rg, eclf], ['Random Forest', 'Extra Trees', 'KNeighbors', 'SVC', 'Ridge Classifier', 'Ensemble']):
    scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))


Accuracy: 0.58 (+/- 0.07) [Random Forest]
Accuracy: 0.58 (+/- 0.05) [Extra Trees]
Accuracy: 0.50 (+/- 0.07) [KNeighbors]
Accuracy: 0.54 (+/- 0.01) [SVC]
Accuracy: 0.56 (+/- 0.03) [Ridge Classifier]
Accuracy: 0.57 (+/- 0.05) [Ensemble]
